In [ ]:
# 필요한 라이브러리 설치
!pip install -qU pymupdf4llm transformers sentence-transformers datasets bitsandbytes
!pip install orjson==3.9.15
!pip install -qU langchain chromadb langchain-chroma langchain_community
!pip install langchain-huggingface

In [ ]:
# 필요한 모듈 임포트
import pymupdf4llm
import re
from tqdm.notebook import tqdm
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from transformers import AutoTokenizer
import os
import pandas as pd
import unicodedata
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# PDF 파일을 처리하고 분할하는 함수
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 파일을 읽고 지정된 크기로 분할"""
    pdf = pymupdf4llm.to_markdown(file_path)  # PDF 파일을 마크다운 형식으로 변환
    pdf = re.sub(r'(?<!\n)\n\n(?!\n)', '\n', pdf)  # 줄바꿈 관련 전처리
    doc = [Document(page_content=pdf, metadata={"source": file_path})]  # Document 객체 생성
    splitter = RecursiveCharacterTextSplitter(  # 문서 분할 설정
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_documents(doc)  # 문서 분할
    return chunk_temp  # 분할된 문서 반환

# 벡터 DB 생성 함수
def create_vector_db(document, embeddings, file_path):
    """문서로부터 벡터 DB 생성"""
    db = Chroma.from_documents(
        documents=document,
        embedding=embeddings,
        persist_directory=file_path
    )
    return db

# PDF 파일들로부터 벡터 DB 생성 함수
def create_pdf_databases(base_directory, embeddings, chunk_size, chunk_overlap):
    """PDF 파일들로부터 각각의 벡터 DB를 생성하고 저장"""
    pdf_databases = {}
    for p in tqdm(os.listdir(base_directory), desc="Creating Vectordb for each PDF"):
        print(p)
        doc = process_pdf(base_directory + p, chunk_size, chunk_overlap)

        # 파일 경로 설정 (train_source 또는 test_source)
        if base_directory.split('/')[-2] == 'train_source':
            file_path = base_directory[:base_directory.find('train_source/')]
        elif base_directory.split('/')[-2] == 'test_source':
            file_path = base_directory[:base_directory.find('test_source/')]

        # 벡터 DB 저장 경로 설정
        vectordb_path = file_path + 'pymupdf4llm/' + p[:p.find('.pdf')]

        # 벡터 DB가 이미 존재하는 경우 로드, 없으면 새로 생성
        if os.path.exists(vectordb_path):
            print(f"Vector DB already exists at {vectordb_path}, loading existing DB...")
            vectordb = Chroma(persist_directory=vectordb_path, embedding_function=embeddings)
        else:
            print(f"Creating new Vector DB at {vectordb_path}...")
            vectordb = create_vector_db(doc, embeddings, vectordb_path)

        # 데이터베이스와 문서 저장
        pdf_databases[unicodedata.normalize('NFC', p.split('.pdf')[0])] = {
            'db': vectordb,
            'doc': doc
        }

    return pdf_databases

In [ ]:
# HuggingFace 임베딩 모델 설정
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# PDF 파일들이 저장된 디렉토리 경로
base_directory = '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/train_source/'

# PDF 파일들로부터 벡터 DB 생성
pdf_databases = create_pdf_databases(base_directory, bge_embeddings, chunk_size=500, chunk_overlap=50)

# 학습용 데이터 CSV 파일 로드
train_df = pd.read_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/train.csv')

# 문서 재정렬을 위한 HuggingFace 모델 설정
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 검색된 문서의 내용을 하나의 문단으로 합치는 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# 토크나이저 설정
model_id = "I-BRICKS/Cerebro_BM_solar_v01"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

# 프롬프트 템플릿 생성
prompt_list = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Creating prompt template"):
    # 각 질문에 대해 상위 5개의 문서 검색
    retriever = pdf_databases[unicodedata.normalize('NFC', row['Source'])]['db'].as_retriever(search_kwargs={"k": 5})

    # 문서 압축을 위한 CrossEncoder 설정
    compressor = CrossEncoderReranker(model=reranker_model, top_n=2)

    # 문서 압축 검색기 초기화
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=retriever
    )

    # 질문에 맞는 문서 검색
    docs = compression_retriever.invoke(row['Question'])
    context = format_docs(docs)
    question = row['Question']
    answer = row['Answer']

    # 프롬프트 템플릿 작성
    template = f"""
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    주어진 질문에만 답변하세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
    답변: {answer}{tokenizer.eos_token}
    """
    prompt_list.append(template)

In [ ]:
# 생성된 프롬프트 리스트를 데이터프레임에 추가
train_df['prompt'] = prompt_list

# 새로운 CSV 파일로 저장
train_df.to_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/finetuning_prompt_cerebro.csv', index=False)